In [1]:
%pip install simpy

In [2]:
# Agregamos impaciencia a la fila.

# Llegan clientes a un sistema de acuerdo a una distribución exponencial negativa
# de media 10 minutos. Allí son atendidos por una persona que sólo puede atender
# a un cliente por vez. Dicha persona demora un tiempo que responde a una distribución
# normal de media 12 minutos y desvío estandar de 1 minuto.
# Los clientes tienen impaciencia, esto quiere decir que se van si no son atendidos
# luego de un determinado tiempo, que para el caso de estudio es 2 +/- 1 minuto.
# Simular la llegada al sistema de 10 clientes.

import random
import simpy

SEMILLA = 42
CANT_CLIENTES = 10
INTERVALO_CLIENTES = 10
MIN_PACIENCIA = 1
MAX_PACIENCIA = 3

#subsistema de generación de clientes
def sistema(env, cantidad, intervalo, empleado):
    for i in range(cantidad):
        proxCliente = cliente(env, i, empleado, tiempoAtencion=12)
        env.process(proxCliente)
        tProxCliente = random.expovariate(1.0 / intervalo)
        yield env.timeout(tProxCliente)

#subsistema de atención de cada cliente (cola+impaciencia+atención)
def cliente(env, nombre, empleado, tiempoAtencion):
    tArribo = env.now
    print(f'Cliente {nombre} llegó a las {tArribo:.2f}')

    with empleado.request() as req:
        tPaciencia = random.uniform(MIN_PACIENCIA, MAX_PACIENCIA)
        resultado = yield req | env.timeout(tPaciencia)

        tEspera = env.now - tArribo

        if req in resultado:
            print(f'Cliente {nombre} fue atendido a las {env.now:.2f} y esperó {tEspera:.2f}')
            tAtencion = random.normalvariate(mu=12, sigma=1)
            yield env.timeout(tAtencion)
            print(f'Cliente {nombre} terminó de ser antendido a las {env.now:.2f}')

        else:
            print(f'Cliente {nombre} se fue por impaciencia a las {env.now:.2f} luego de esperar {tEspera:.2f}')


random.seed(SEMILLA)
env = simpy.Environment()

empleado = simpy.Resource(env, capacity=1)
env.process(sistema(env, CANT_CLIENTES, INTERVALO_CLIENTES, empleado))
env.run()

Cliente 0 llegó a las 0.00
Cliente 0 fue atendido a las 0.00 y esperó 0.00
Cliente 0 terminó de ser antendido a las 3.86
Cliente 1 llegó a las 10.20
Cliente 1 fue atendido a las 10.20 y esperó 0.00
Cliente 2 llegó a las 12.73
Cliente 2 se fue por impaciencia a las 13.90 luego de esperar 1.17
Cliente 1 terminó de ser antendido a las 23.75
Cliente 3 llegó a las 35.00
Cliente 3 fue atendido a las 35.00 y esperó 0.00
Cliente 3 terminó de ser antendido a las 37.96
Cliente 4 llegó a las 40.48
Cliente 4 fue atendido a las 40.48 y esperó 0.00
Cliente 4 terminó de ser antendido a las 43.14
Cliente 5 llegó a las 47.52
Cliente 5 fue atendido a las 47.52 y esperó 0.00
Cliente 5 terminó de ser antendido a las 50.51
Cliente 6 llegó a las 58.01
Cliente 6 fue atendido a las 58.01 y esperó 0.00
Cliente 6 terminó de ser antendido a las 58.09
Cliente 7 llegó a las 66.91
Cliente 7 fue atendido a las 66.91 y esperó 0.00
Cliente 7 terminó de ser antendido a las 71.90
Cliente 8 llegó a las 83.30
Cliente 8 fu